<처음 코드가 수행이 안되었던 이유! >


(1) `Font not found ` 
* 운영체제에도 해당 폰트가 있어야 한다.
* 권한 문제로 수정이 안 되는 경우 matplotlibrc 파일 속성에서 사용 권한 수정
* ref) https://jinyes-tistory.tistory.com/70


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)

# 그래프에서 폰트가 안보이는 경우
#matplotlib.font_manager._rebuild()

%matplotlib inline

from IPython.display import set_matplotlib_formats

# ipython 7.23 이후 set_matplotlib_formats 함수가 사라짐
%config InlineBackend.figure_format = 'retina'

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# gdrive에서 파일 불러오기
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/med_201909.csv', encoding = 'utf-8')
df.shape


In [ ]:
df.head(1)

# Q1 전국 시도별 약국 수

* 상권업종소분류명이 약국인 것을 찾아 빈도수를 구해주세요.
* value_counts, groupby, pivot_table 등 다양한 집계 방법 사용

In [ ]:
df[df['상권업종소분류명']=='약국']['시도명'].value_counts()

## Q2 동물병원도 병원이나 약국이 많은 곳에 더 많이 있을까?

In [ ]:
df_animal = df[df['상권업종소분류명']=='동물병원']['시도명'].value_counts()
df_animal

In [ ]:
df_animal.plot.barh(figsize=(9,8),grid=True)
plt.show()

## Q3 해당 지역에 피부과나 성형외과가 다른 지역에 비해 전체 병원 수 대비 어느 정도 비율 차지하고 있는지?

* ['상권업종소분류명'] 중 "피부" 나 "성형"이 들어간 분류명 찾아보기 
* 피부과 성형외과 수 / 전체 병원 수 비율
* 비율이 높은 순서대로
* 소수점 두 번째 자리까지 출력하는 방법 : pandas round 

In [ ]:
# 서울특별시에 있는 병원 색인 => 서울특별시 & 병원
df_seoul_med = df[(df["시도명"]=="서울특별시") & (df["상권업종중분류명"]=="병원")].copy()

# 서울특별시 병원 중 '피부', '성형' 들어간 분류명
df_seoul_med_sur=df_seoul_med[df_seoul_med["상권업종소분류명"].str.contains("피부|성형")].copy()

# 피부과+성형외과 / 전체병원
rat = df_seoul_med_sur["시군구명"].value_counts()/df_seoul_med["시군구명"].value_counts()

# 소수점 둘째자리까지 출력 -> 
# (1) sort_values() 사용하여 비율이 높은 순서 정렬 -> ascending=False
# (2) pandas round 사용
rat.sort_values(ascending=False).round(2)

## Q4. 지도 시각화 -> '노인/치매병원'이 주로 어디에 위치하고 있는지!
* folium 사용하여 전국의 '노인/치매병원' 표시

In [ ]:
# folium 설치
!pip install --target=$my_path folium

In [ ]:
import folium

In [ ]:
silver = df[df['상권업종소분류명']=='노인/치매병원']
map = folium.Map(location=[silver["위도"].mean(),silver["경도"].mean()], 
                 zoom_start=12, tiles='Stamen Terrain',
                 scrollWheelZoom=False)

for n in silver.index:
    name = silver.loc[n, "상호명"]
    address = silver.loc[n, "도로명주소"]
    popup = f"{name}-{address}"
    location = [silver.loc[n, "위도"], silver.loc[n, "경도"]]
    folium.Marker(
        location = location,
        popup = popup,
        icon=folium.Icon(color="blue", icon="bookmark")
    ).add_to(map)
map
